### 安裝mysql套件


In [112]:
# pip install pymysql

In [113]:
import pymysql
pymysql.__version__

'1.4.6'

### 連接mysql

In [155]:
connect=pymysql.connect(
    user='root',
    passwd='12345678',
    host='127.0.0.1',
    database='shop',
    port=3306
)
connect

In [156]:
cursor=connect.cursor()
cursor

### 查詢語法
- select * from tablename where ...

In [130]:
sqlstr="select id,email,phone,address from user where id=1;"

cursor.execute(sqlstr)
users=cursor.fetchone()
users

(1, 'aaa@gmail.com', '0123456789', '台北市內湖區內湖路一段')

### 註冊功能

In [117]:
username=input("姓名:")
email=input("email:")
password=input("passwd")
phone=input("phone:")
address=input("address:")

sqlstr=f"insert into user (username,email,password,phone,address)\
      values ('{username}','{email}','{password}','{phone}','{address}');"

sqlstr

"insert into user (username,email,password,phone,address)      values ('','','','','');"

In [118]:
try:
    cursor.execute(sqlstr)
    connect.commit()
except Exception as e:
    print("寫入失敗",e)

### 進行登入

In [131]:
email=input("email:")
password=input("passwd")

sqlstr=f"select * from user where email='{email}' and password='{password}'"
sqlstr

"select * from user where email='bbb@gmail.com' and password='12345678'"

In [132]:
cursor.execute(sqlstr)
user=cursor.fetchone()
user

(2,
 None,
 '12345678',
 'bbb@gmail.com',
 '0978545714',
 '',
 datetime.datetime(2024, 8, 24, 10, 35, 22))

In [133]:
if user is not None:
    print("登入成功")
else:
    print("登入失敗")

登入成功


### 建立商品資料表(item)
- id            - int(AI)
- name          - varchar(100) - unique
- price         - float
- qty           - int          - default:1
- info          -text          - 可以為空

### 商品列表

In [134]:
def update_items():
    sqlstr="select * from item;"
    cursor.execute(sqlstr)
    items=cursor.fetchall()
    # items
    return items

update_items()

((1, 'apple', 100.0, 0, 'fresh apple', datetime.date(2024, 8, 24)),
 (2, 'pen', 20.0, 95, 'good quality pen', datetime.date(2024, 8, 24)),
 (3, 'banana', 25.5, 9996, None, datetime.date(2024, 8, 24)),
 (4, 'ball', 500.5, 0, None, datetime.date(2024, 8, 24)))

In [135]:
len(items)

4

In [137]:
cart=[]


while True: 
    #同步更新最新產品
    items=update_items()
    #選單
    print("-"*100)
    for i ,item in enumerate(items):
        # print(item)
        print(f"{i+1}.商品名稱:{item[1]} 價格:{item[2]} 數量:{item[3]} 備註:{item[4]}")
    print("-"*100)


    choice=int(input("輸入編號(0:exit):"))
    if choice==0:
        break

    # 選項<1 || >len(item) 輸入不正確
    if choice<1 or choice>len(items):
        print("輸入錯誤")
        continue

    # 紀錄購買商品
    buy_item=items[choice-1]
    if buy_item[3]<1:
        print(f"****{buy_item[1]} 無庫存")
        continue

    #加入購物車
    cart.append(buy_item)

    #更新商品數量
    sqlstr=f"update item set qty={buy_item[3]-1} where id={buy_item[0]};"
    print(sqlstr)
    cursor.execute(sqlstr)
    connect.commit()


    print(f"===>{buy_item[1]} 加入購物車")

print("完成購買")
print(cart)

----------------------------------------------------------------------------------------------------
1.商品名稱:apple 價格:100.0 數量:0 備註:fresh apple
2.商品名稱:pen 價格:20.0 數量:95 備註:good quality pen
3.商品名稱:banana 價格:25.5 數量:9996 備註:None
4.商品名稱:ball 價格:500.5 數量:0 備註:None
----------------------------------------------------------------------------------------------------
update item set qty=94 where id=2;
===>pen 加入購物車
----------------------------------------------------------------------------------------------------
1.商品名稱:apple 價格:100.0 數量:0 備註:fresh apple
2.商品名稱:pen 價格:20.0 數量:94 備註:good quality pen
3.商品名稱:banana 價格:25.5 數量:9996 備註:None
4.商品名稱:ball 價格:500.5 數量:0 備註:None
----------------------------------------------------------------------------------------------------
update item set qty=93 where id=2;
===>pen 加入購物車
----------------------------------------------------------------------------------------------------
1.商品名稱:apple 價格:100.0 數量:0 備註:fresh apple
2.商品名稱:pen 價格:20.0 數量:93 備註:good qual

### 送出訂單
- id
- user_id
- item_id
- created_time

In [138]:
# cart
for item in cart:
    # print(item)
    sqlstr=f"insert into `order` (user_id,item_id) values ({user[0]},{item[0]})"
    # print(sqlstr)
    cursor.execute(sqlstr)
    connect.commit()
    print("送出訂單")

送出訂單
送出訂單
送出訂單


### 開始出貨

In [139]:
# 找出user

sqlstr="select * from `order`;"
cursor.execute(sqlstr)
orders=cursor.fetchall()
orders

((5, 1, 2, datetime.datetime(2024, 8, 24, 14, 58, 32)),
 (6, 1, 2, datetime.datetime(2024, 8, 24, 14, 58, 32)),
 (7, 2, 2, datetime.datetime(2024, 8, 24, 15, 25, 7)),
 (8, 2, 2, datetime.datetime(2024, 8, 24, 15, 25, 7)),
 (9, 2, 2, datetime.datetime(2024, 8, 24, 15, 25, 7)))

In [140]:
user_id=orders[0][1]
user_id

1

In [141]:
sqlstr=f"select * from user where id={user_id};"
cursor.execute(sqlstr)
user=cursor.fetchone()
user

(1,
 None,
 '12345678',
 'aaa@gmail.com',
 '0123456789',
 '台北市內湖區內湖路一段',
 datetime.datetime(2024, 8, 24, 10, 10, 56))

In [157]:
sqlstr=f"select id,item_id from `order` where user_id={user_id};"
cursor.execute(sqlstr)
buy_items=cursor.fetchall()
buy_items

((5, 2), (6, 2))

In [158]:
"""
收貨人:
email:
phone:
address:

=========================
1.pen 20
2.pen 20

總計:
=========================


"""
print(f"收貨人:{user[1]}")
print(f"email:{user[3]}")
print(f"phone:{user[4]}")
print(f"address:{user[5]}")
print("="*50)

total_price=0
for i,item_id in enumerate(buy_items):
    # print(item_id[0])
    sqlstr=f"select name,price from item where id={item_id[1]};"
    cursor.execute(sqlstr)
    item=cursor.fetchone()
    # print(item)
    total_price+=item[1]
    print(f"{i+1}.{item[0]} price:{item[1]}")


print(f"總計:{total_price}")
print("="*50)    

收貨人:None
email:aaa@gmail.com
phone:0123456789
address:台北市內湖區內湖路一段
1.pen price:20.0
2.pen price:20.0
總計:40.0


### 將以出貨訂單，增加欄位shipped(Bool => true/false)

In [162]:
for order_id in buy_items:
    print(order_id[0])
    sqlstr=f"update `order` set shipped=true where id={order_id[0]};"
    cursor.execute(sqlstr)
    connect.commit()


5
6


In [152]:
connect.close()